In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [4]:
Today = '2024-04-30'

print_to_excel = True

In [8]:
query = f"""
select * 
from xxwh."OPRECConso_detailed"
where "periodDate" = TO_DATE('{Today}', 'YYYY-MM-DD')
and "rowType" != 'on hand'
"""
data_OPRECConso_detailed = export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:172: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [62]:
data_OPRECConso_detailed_work = data_OPRECConso_detailed[['businessUnit','customer','appliedPrice','dealPriceDealCurrency','currency','appliedEndDate','quantity','unshippedDealAmount']]
# merge segments
data_OPRECConso_detailed_work['businessSegment'] = merge_SalesUnits(data_OPRECConso_detailed_work,'businessUnit',merge_col='ocpSegment')
data_OPRECConso_detailed_work['customerCode'] = merge_Mapping(data_OPRECConso_detailed_work,'customer')
data_OPRECConso_detailed_work['customerSegment'] = merge_SalesUnits(data_OPRECConso_detailed_work,'customerCode',merge_col='ocpSegment')
# work with data
data_OPRECConso_detailed_work_not_na = data_OPRECConso_detailed_work[data_OPRECConso_detailed_work.unshippedDealAmount!=0].drop('quantity', axis=1).dropna(subset='appliedPrice').reset_index(drop=True)
data_OPRECConso_detailed_work_not_na['Amount_appliedPrice'] = data_OPRECConso_detailed_work_not_na.appliedPrice*data_OPRECConso_detailed_work_not_na.unshippedDealAmount
data_OPRECConso_detailed_work_not_na['Amount_dealPriceDealCurrency'] = data_OPRECConso_detailed_work_not_na.dealPriceDealCurrency*data_OPRECConso_detailed_work_not_na.unshippedDealAmount
data_OPRECConso_detailed_work_not_na = Period(data_OPRECConso_detailed_work_not_na, Today, 'appliedEndDate')
### to excel
if print_to_excel==True:
    data_print = data_OPRECConso_detailed_work_not_na[['businessSegment','businessUnit','customerSegment','currency','Amount_appliedPrice','Amount_dealPriceDealCurrency','appliedEndDate','Days','Period']]
    Output_file = f'{str(date.today())}_ECH_deals_OPRECConso_detailed.xlsx'
    data_print.to_excel(Output_file, index=False)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['RUFERT' 'RUFERT' 'RUFERT' ... 'EURSALES' 'RUFERT' 'RUFERT']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11416\1837765220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [66]:
mapping_data = data_OPRECConso_detailed_work.loc[data_OPRECConso_detailed_work.customerSegment=='External',['customer','customerCode','customerSegment']].drop_duplicates()
new_list(mapping_data,Output_file,'mapping_sheet')